In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [84]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

In [85]:
df_kerry = pd.read_csv('./csv_files/speech_kerry')
df_bush = pd.read_csv('./csv_files/speech_bush')
df_obama = pd.read_csv('./csv_files/speech_obama')
df_mccain = pd.read_csv('./csv_files/speech_mccain')
df_obama_2 = pd.read_csv('./csv_files/speech_obama_2')
df_romney = pd.read_csv('./csv_files/speech_romney')
df_clinton = pd.read_csv('./csv_files/speech_clinton')
df_trump = pd.read_csv('./csv_files/speech_trump')

In [86]:
df_kerry['party'] = 'dem'
df_bush['party'] = 'rep'
df_obama['party'] = 'dem'
df_mccain['party'] = 'rep'
df_obama_2['party'] = 'dem'
df_romney['party'] = 'rep'
df_clinton['party'] = 'dem'
df_trump['party'] = 'rep'

In [87]:
df = pd.concat([df_kerry, df_bush, df_obama, df_mccain, df_obama_2, df_romney, 
                df_clinton, df_trump], axis=0, ignore_index=True)

In [88]:
df.reindex()
df.shape

(1004, 4)

# Preprocessing

In [89]:
df['speech'] = df['speech'].apply(lambda x: x.replace("\'", "'"))
df['speech'] = df['speech'].apply(lambda x: x.lower().replace("kerry", ""))
df['speech'] = df['speech'].apply(lambda x: x.lower().replace("bush", ""))
df['speech'] = df['speech'].apply(lambda x: x.lower().replace("obama", ""))
df['speech'] = df['speech'].apply(lambda x: x.lower().replace("mccain", ""))
df['speech'] = df['speech'].apply(lambda x: x.lower().replace("romeny", ""))
df['speech'] = df['speech'].apply(lambda x: x.lower().replace("clinton", ""))
df['speech'] = df['speech'].apply(lambda x: x.lower().replace("trump", ""))

In [90]:
df.party = df.party.apply(lambda x: 1 if x == 'dem' else 0)
X = df.speech 
y = df.party

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

In [100]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(X_train)
X_train_tf_tsvd = tfidf.transform(X_train)

# tsvd = TruncatedSVD(n_components=100)
# tsvd.fit(X_train_tf)
# X_train_tf_tsvd = tsvd.transform(X_train_tf)

rfc = RandomForestClassifier(n_estimators=100, max_depth=5)
rfc.fit(X_train_tf_tsvd, y_train)
train_predictions = rfc.predict(X_train_tf_tsvd)
print(rfc.score(X_train_tf_tsvd, y_train))
print(confusion_matrix(y_train, train_predictions))
print(classification_report(y_train, train_predictions))

X_test_tf_tsvd = tfidf.transform(X_test)
# X_test_tf_tsvd = tsvd.transform(X_test_tf)
test_predictions = rfc.predict(X_test_tf_tsvd)
print(rfc.score(X_test_tf_tsvd, y_test))
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)>
0.950166112957
[[204  30]
 [  0 368]]
             precision    recall  f1-score   support

          0       1.00      0.87      0.93       234
          1       0.92      1.00      0.96       368

avg / total       0.95      0.95      0.95       602

0.833333333333
[[ 89  65]
 [  2 246]]
             precision    recall  f1-score   support

          0       0.98      0.58      0.73       154
          1       0.79      0.99      0.88       248

avg / total       0.86      0.83      0.82    

In [99]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(X_train)
X_train_tf = tfidf.transform(X_train)

tsvd = TruncatedSVD(n_components=100)
tsvd.fit(X_train_tf)
X_train_tf_tsvd = tsvd.transform(X_train_tf)

rfc = RandomForestClassifier(n_estimators=100, max_depth=5)
rfc.fit(X_train_tf_tsvd, y_train)
train_predictions = rfc.predict(X_train_tf_tsvd)
print(rfc.score(X_train_tf_tsvd, y_train))
print(confusion_matrix(y_train, train_predictions))
print(classification_report(y_train, train_predictions))

X_test_tf = tfidf.transform(X_test)
X_test_tf_tsvd = tsvd.transform(X_test_tf)
test_predictions = rfc.predict(X_test_tf_tsvd)
print(rfc.score(X_test_tf_tsvd, y_test))
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

0.961794019934
[[211  23]
 [  0 368]]
             precision    recall  f1-score   support

          0       1.00      0.90      0.95       234
          1       0.94      1.00      0.97       368

avg / total       0.96      0.96      0.96       602

0.830845771144
[[ 88  66]
 [  2 246]]
             precision    recall  f1-score   support

          0       0.98      0.57      0.72       154
          1       0.79      0.99      0.88       248

avg / total       0.86      0.83      0.82       402



In [101]:
cv = CountVectorizer()
cv.fit(X_train)
cv.transform(X_train)

cv.get_feature_names()


['00',
 '000',
 '0000',
 '01',
 '02',
 '025',
 '026',
 '03',
 '04',
 '05',
 '050',
 '055',
 '06',
 '07',
 '08',
 '09',
 '10',
 '100',
 '1000',
 '1009',
 '100th',
 '101',
 '101st',
 '102',
 '103',
 '104',
 '105',
 '105th',
 '106',
 '107',
 '1079',
 '109',
 '10th',
 '11',
 '110',
 '110th',
 '111434',
 '112',
 '114',
 '115',
 '116',
 '116th',
 '117',
 '118',
 '1189',
 '11th',
 '12',
 '120',
 '1200',
 '1222',
 '123',
 '123rd',
 '124th',
 '125',
 '126th',
 '129th',
 '12th',
 '13',
 '130',
 '1300',
 '130th',
 '134',
 '139',
 '13th',
 '14',
 '140',
 '1400',
 '143',
 '144',
 '145',
 '147',
 '148',
 '14d',
 '14th',
 '15',
 '150',
 '1500',
 '152',
 '155',
 '158',
 '159',
 '15th',
 '16',
 '160',
 '1600',
 '161',
 '1630s',
 '164',
 '165',
 '168',
 '16th',
 '17',
 '170',
 '1700',
 '173',
 '174',
 '175',
 '1776',
 '1777',
 '1778',
 '1787',
 '1789',
 '1791',
 '17th',
 '18',
 '180',
 '1800',
 '1800s',
 '1808',
 '1827',
 '183',
 '1842',
 '1848',
 '185',
 '1850',
 '1858',
 '186',
 '1860',
 '1869',
 '187

In [102]:
lda = LatentDirichletAllocation(n_components=5)
lda.fit(X_train)

ValueError: could not convert string to float: 'todd: we\'ll have my interview this morning with donald  in just a few minutes. but we\'re going to begin with the democrats and former secretary of state, hillary . you liked that "graveyard of pollsters" line, didn\'t you? : well, i\'ve lived through this twice — todd: you\'ve lived through this. : — in \'92 and 2008. and look, i know i\'m behind. and i\'m going to keep fighting until the very last vote is counted because i care about this primary. todd: why do you think you started with such a big deficit this time, considering — look, this has been a big state to both your husband and yourself, very important. : yes. todd: they\'ve always helped you. : well, look, i can\'t sit here and analyze all the reasons. obviously, one is that the senator has been in public life next door for 25 years. so there\'s a familiarity with him. i totally respect that. and i just think that what\'s great about this primary is new hampshire voters take a

In [69]:
results = pd.DataFrame(lda.components_,
                      columns=feature_names)

AttributeError: 'LatentDirichletAllocation' object has no attribute 'components_'